In [1]:
# Download of the installation packages for Geopy API
!conda install -c conda-forge geopy --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [2]:
# Download of the installation packages for Folium API
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         560 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


branca-0.3.1         | 25 KB     | #############################

# Import of the libraries used:

In [4]:
# import libraries
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim
import folium 

In [5]:
#FOURSQUARE IDS
CLIENT_ID = 'Z0J03ZIUK0D4NGFQ5BFKEWUOIZASGRALFMXLBZAAUUHMLY4U' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20190606'
LIMIT = 50

# GPS Coordinates of our locations

Our three locations being reviewed are:
- Calle de Viriato 43
- Paseo de la Castellana, 259D
- Calle de Pradillo 19

Let's get the GPS coordinates of each address.

In [6]:
#Address: Calle de Viriato 43
address_01 = 'Calle de Viriato 43, Madrid, Spain'
geolocator_01 = Nominatim(user_agent="foursquare_agent")
location_01 = geolocator_01.geocode(address_01)
latitude_01 = location_01.latitude
longitude_01 = location_01.longitude
print(latitude_01, longitude_01)

search_query_01 = 'Italian'
radius_01 = 2000
print(search_query_01 + ' .... OK!')

40.4359867 -3.7033085
Italian .... OK!


# Query of Foursquare API using our ids:

We will get results as a JSON object

In [7]:
url_01 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_01, longitude_01, VERSION, search_query_01, radius_01, LIMIT)
url_01

'https://api.foursquare.com/v2/venues/search?client_id=Z0J03ZIUK0D4NGFQ5BFKEWUOIZASGRALFMXLBZAAUUHMLY4U&client_secret=DH4O4SF3KKT4GQGHETVCKFUST0LU4LFC2J34KRGPZSDWBOKD&ll=40.4359867,-3.7033085&v=20190606&query=Italian&radius=2000&limit=50'

Let's take the results into json-object, so we can move it to dataframe:

In [8]:
results_01 = requests.get(url_01).json()
results_01

{'meta': {'code': 200, 'requestId': '5d079a376adbf5002c701a17'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/embassy_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d12c951735',
      'name': 'Embassy / Consulate',
      'pluralName': 'Embassies / Consulates',
      'primary': True,
      'shortName': 'Embassy'}],
    'hasPerk': False,
    'id': '4c0958e3ffb8c9b6bdc26961',
    'location': {'address': 'Calle de Lagasca, 98',
     'cc': 'ES',
     'city': 'Madrid',
     'country': 'España',
     'distance': 1648,
     'formattedAddress': ['Calle de Lagasca, 98',
      '28006 Madrid Madrid',
      'España'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.43260015265081,
       'lng': -3.684363892032192}],
     'lat': 40.43260015265081,
     'lng': -3.684363892032192,
     'postalCode': '28006',
     'state': 'Madrid'},
    'name': 'Embajada de la República Italiana - Ambasciata della Repubblica Italiana

In [9]:
# assign relevant part of JSON to venues
venues_01 = results_01['response']['venues']

# tranform venues into a dataframe
from pandas.io.json import json_normalize
dataframe_01 = json_normalize(venues_01)

#number of entries:
dataframe_01.shape

(22, 18)

# We will review our data, so only includes the useful information

In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_01 = ['name', 'categories'] + [col for col in dataframe_01.columns if col.startswith('location.')] + ['id']
dataframe_filtered_01 = dataframe_01.loc[:, filtered_columns_01]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_01['categories'] = dataframe_filtered_01.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_01.columns = [column.split('.')[-1] for column in dataframe_filtered_01.columns]

dataframe_filtered_01

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Embajada de la República Italiana - Ambasciata...,Embassy / Consulate,"Calle de Lagasca, 98",ES,Madrid,España,NaN,1648,"[Calle de Lagasca, 98, 28006 Madrid Madrid, Es...","[{'label': 'display', 'lat': 40.43260015265081...",40.432600,-3.684364,28006,Madrid,4c0958e3ffb8c9b6bdc26961
1,iS Italian Skills,Boutique,"C. Gravina, 9",ES,Madrid,España,NaN,1515,"[C. Gravina, 9, 28004 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.42306918055597...",40.423069,-3.697672,28004,Madrid,52ed42bc11d28180d18d8944
2,Mercato Italiano,Food & Drink Shop,"C. Rios Rosas, 50",ES,Madrid,España,C. Modesto Lafuente,880,"[C. Rios Rosas, 50 (C. Modesto Lafuente), 2800...","[{'label': 'display', 'lat': 40.44163816155172...",40.441638,-3.696035,28003,Madrid,584ac020ec980f4c6bc8b35f
3,italiana_madrid,Café,"Corredera Baja de San Pablo,10",ES,Madrid,España,NaN,1543,"[Corredera Baja de San Pablo,10, 28004 Madrid ...","[{'label': 'display', 'lat': 40.42213863996650...",40.422139,-3.704285,28004,Madrid,509d4c9ee4b05e06b4409edd
4,Matteo Cucina Italiana,Italian Restaurant,Calle Ayala 28,ES,Madrid,España,NaN,1792,"[Calle Ayala 28, 28001 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.42705440180079...",40.427054,-3.685701,28001,Madrid,532d7469498e6f7a11b53ddc
5,Liceo Italiano Enrico Fermi,School,NaN,ES,Madrid,España,NaN,940,"[Madrid Madrid, España]","[{'label': 'display', 'lat': 40.4419026679497,...",40.441903,-3.695376,NaN,Madrid,4e310c00d1643f195b90e579
6,Cámara de Comercio e Industria Italiana para E...,Non-Profit,"Calle Cristobal Bordiú, 54",ES,Madrid,España,NaN,1017,"[Calle Cristobal Bordiú, 54, 28002 Madrid Madr...","[{'label': 'display', 'lat': 40.44274209763634...",40.442742,-3.695215,28002,Madrid,519dcef6454a4051f588105c
7,Camara de comercio italiana madrid,Non-Profit,"Calle Cristobal Bordiú, 54 MADRID",ES,Madrid,España,NaN,1026,"[Calle Cristobal Bordiú, 54 MADRID, 28002 Madr...","[{'label': 'display', 'lat': 40.44282374988759...",40.442824,-3.695183,28002,Madrid,519dcc9d454a4051f5866413
8,Bellamia Gelateria Italiana,Ice Cream Shop,"C. Pérez Galdós, 3",ES,"Madrid Centro, Madrid",España,"C. Fuencarral, 34",1526,"[C. Pérez Galdós, 3 (C. Fuencarral, 34), 28004...","[{'label': 'display', 'lat': 40.42244828094222...",40.422448,-3.700408,28004,Madrid,5390ee01498e87f1a952ff0e
9,Capperi Mercato Italiano,Deli / Bodega,C/ Fernando VI 2,ES,Madrid,España,NaN,1238,"[C/ Fernando VI 2, Madrid Madrid, España]","[{'label': 'display', 'lat': 40.42590349266851...",40.425903,-3.697125,NaN,Madrid,4b62d14bf964a520ef532ae3


In [12]:
dataframe_filtered_01.name

0     Embajada de la República Italiana - Ambasciata...
1                                     iS Italian Skills
2                                      Mercato Italiano
3                                       italiana_madrid
4                                Matteo Cucina Italiana
5                           Liceo Italiano Enrico Fermi
6     Cámara de Comercio e Industria Italiana para E...
7                    Camara de comercio italiana madrid
8                           Bellamia Gelateria Italiana
9                              Capperi Mercato Italiano
10                                      Cucina italiana
11                       Aperitivo Italiano en Malasaña
12                        Instituto Italiano de Cultura
13                             Heladeria Salon Italiano
14                                   Il Centro Italiano
15                              4D Caffetteria Italiana
16                                        Café italiano
17                          Pizzeria Italiana Ma

# We will use the Folium to plot our dataframe, based on each venue location

In [17]:
venues_map_01 = folium.Map(location=[latitude_01, longitude_01], zoom_start=14) # generate map centred around the Conrad Hotel

# add a red circle marker to represent
folium.features.CircleMarker(
    [latitude_01, longitude_01],
    radius=10,
    color='red',
    popup='Our venue spot',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_01)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_01.lat, dataframe_filtered_01.lng, dataframe_filtered_01.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_01)

# display map
venues_map_01

In [15]:
#Let's save the map as html:
venues_map_01.save('v_01.html')

# Repeat the steps performed for locations 2, and 3:

In [18]:
#Address: Paseo de la Castellana, 259D
address_02 = 'Paseo de la Castellana, 259D, Madrid, Spain'
geolocator_02 = Nominatim(user_agent="foursquare_agent")
location_02 = geolocator_02.geocode(address_02)
latitude_02 = location_02.latitude
longitude_02 = location_02.longitude
print(latitude_02, longitude_02)

search_query_02 = 'Italian'
radius_02 = 3000
print(search_query_02 + ' .... OK!')

40.4790377 -3.6868739
Italian .... OK!


In [19]:
url_02 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_02, longitude_02, VERSION, search_query_02, radius_02, LIMIT)
url_02

'https://api.foursquare.com/v2/venues/search?client_id=Z0J03ZIUK0D4NGFQ5BFKEWUOIZASGRALFMXLBZAAUUHMLY4U&client_secret=DH4O4SF3KKT4GQGHETVCKFUST0LU4LFC2J34KRGPZSDWBOKD&ll=40.4790377,-3.6868739&v=20190606&query=Italian&radius=3000&limit=50'

In [21]:
results_02 = requests.get(url_02).json()
results_02

{'meta': {'code': 200, 'requestId': '5d079fac1ed2194417ca7ddf'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '52e81612bcbc57f1066b79f1',
      'name': 'Bistro',
      'pluralName': 'Bistros',
      'primary': True,
      'shortName': 'Bistro'}],
    'hasPerk': False,
    'id': '53e1367f498ecd25ba071919',
    'location': {'cc': 'ES',
     'city': 'Madrid',
     'country': 'España',
     'distance': 3138,
     'formattedAddress': ['Madrid Madrid', 'España'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.493719,
       'lng': -3.655233}],
     'lat': 40.493719,
     'lng': -3.655233,
     'state': 'Madrid'},
    'name': 'bistro italiano',
    'referralId': 'v-1560780716'}]}}

In [22]:
# assign relevant part of JSON to venues
venues_02 = results_02['response']['venues']

# tranform venues into a dataframe
from pandas.io.json import json_normalize
dataframe_02 = json_normalize(venues_02)

#number of entries:
dataframe_02.shape

(1, 14)

In [23]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_02 = ['name', 'categories'] + [col for col in dataframe_02.columns if col.startswith('location.')] + ['id']
dataframe_filtered_02 = dataframe_02.loc[:, filtered_columns_02]

# function that extracts the category of the venue
def get_category_type_02(row):
    try:
        categories_list_02 = row['categories']
    except:
        categories_list_02 = row['venue.categories']
        
    if len(categories_list_02) == 0:
        return None
    else:
        return categories_list_02[0]['name']

# filter the category for each row
dataframe_filtered_02['categories'] = dataframe_filtered_02.apply(get_category_type_02, axis=1)

# clean column names by keeping only last term
dataframe_filtered_02.columns = [column.split('.')[-1] for column in dataframe_filtered_02.columns]

dataframe_filtered_02

,name,categories,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,bistro italiano,Bistro,ES,Madrid,España,3138,"[Madrid Madrid, España]","[{'label': 'display', 'lat': 40.493719, 'lng':...",40.493719,-3.655233,Madrid,53e1367f498ecd25ba071919


In [24]:
dataframe_filtered_02.name


0    bistro italiano
Name: name, dtype: object

In [25]:
venues_map_02 = folium.Map(location=[latitude_02, longitude_02], zoom_start=14) # generate map centred around the Conrad Hotel

# add a red circle marker to represent
folium.features.CircleMarker(
    [latitude_02, longitude_02],
    radius=10,
    color='red',
    popup='Our venue spot',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_02)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_02.lat, dataframe_filtered_02.lng, dataframe_filtered_02.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_02)

# display map
venues_map_02

In [ ]:
venues_map_02.save('v_02.html')

In [26]:
#Address: Calle de Pradillo 19
address_03 = 'Calle de Pradillo 19, Madrid, Spain'
geolocator_03 = Nominatim(user_agent="foursquare_agent")
location_03 = geolocator_03.geocode(address_03)
latitude_03 = location_03.latitude
longitude_03 = location_03.longitude
print(latitude_03, longitude_03)

search_query_03 = 'Italian'
radius_03 = 2000
print(search_query_03 + ' .... OK!')

40.4488336 -3.6733894
Italian .... OK!


In [27]:
url_03 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_03, longitude_03, VERSION, search_query_03, radius_03, LIMIT)
url_03

'https://api.foursquare.com/v2/venues/search?client_id=Z0J03ZIUK0D4NGFQ5BFKEWUOIZASGRALFMXLBZAAUUHMLY4U&client_secret=DH4O4SF3KKT4GQGHETVCKFUST0LU4LFC2J34KRGPZSDWBOKD&ll=40.4488336,-3.6733894&v=20190606&query=Italian&radius=2000&limit=50'

In [28]:
results_03 = requests.get(url_03).json()
results_03

{'meta': {'code': 200, 'requestId': '5d07a11df594df1b1c73ec2e'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/embassy_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d12c951735',
      'name': 'Embassy / Consulate',
      'pluralName': 'Embassies / Consulates',
      'primary': True,
      'shortName': 'Embassy'}],
    'hasPerk': False,
    'id': '4c0958e3ffb8c9b6bdc26961',
    'location': {'address': 'Calle de Lagasca, 98',
     'cc': 'ES',
     'city': 'Madrid',
     'country': 'España',
     'distance': 2032,
     'formattedAddress': ['Calle de Lagasca, 98',
      '28006 Madrid Madrid',
      'España'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.43260015265081,
       'lng': -3.684363892032192}],
     'lat': 40.43260015265081,
     'lng': -3.684363892032192,
     'postalCode': '28006',
     'state': 'Madrid'},
    'name': 'Embajada de la República Italiana - Ambasciata della Repubblica Italiana

In [29]:
# assign relevant part of JSON to venues
venues_03 = results_03['response']['venues']

# tranform venues into a dataframe
from pandas.io.json import json_normalize
dataframe_03 = json_normalize(venues_03)

#number of entries:
dataframe_03.shape

(9, 17)

In [30]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_03 = ['name', 'categories'] + [col for col in dataframe_03.columns if col.startswith('location.')] + ['id']
dataframe_filtered_03 = dataframe_03.loc[:, filtered_columns_03]

# function that extracts the category of the venue
def get_category_type_03(row):
    try:
        categories_list_03 = row['categories']
    except:
        categories_list_03 = row['venue.categories']
        
    if len(categories_list_03) == 0:
        return None
    else:
        return categories_list_03[0]['name']

# filter the category for each row
dataframe_filtered_03['categories'] = dataframe_filtered_03.apply(get_category_type_03, axis=1)

# clean column names by keeping only last term
dataframe_filtered_03.columns = [column.split('.')[-1] for column in dataframe_filtered_03.columns]

dataframe_filtered_03

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Embajada de la República Italiana - Ambasciata...,Embassy / Consulate,"Calle de Lagasca, 98",ES,Madrid,España,NaN,2032,"[Calle de Lagasca, 98, 28006 Madrid Madrid, Es...","[{'label': 'display', 'lat': 40.43260015265081...",40.432600,-3.684364,28006,Madrid,4c0958e3ffb8c9b6bdc26961
1,Mercato Italiano,Food & Drink Shop,"C. Rios Rosas, 50",ES,Madrid,España,C. Modesto Lafuente,2078,"[C. Rios Rosas, 50 (C. Modesto Lafuente), 2800...","[{'label': 'display', 'lat': 40.44163816155172...",40.441638,-3.696035,28003,Madrid,584ac020ec980f4c6bc8b35f
2,Il Centro Italiano,Student Center,General diaz porlier,ES,Madrid,España,NaN,1222,"[General diaz porlier, Madrid Madrid, España]","[{'label': 'display', 'lat': 40.43893741301658...",40.438937,-3.679639,NaN,Madrid,4d405eeeadb6236aaac58740
3,Il Centro Italiano,Student Center,General diaz porlier,ES,Madrid,España,NaN,1588,"[General diaz porlier, Madrid Madrid, España]","[{'label': 'display', 'lat': 40.43474589197704...",40.434746,-3.676365,NaN,Madrid,4d405eee915a3704c1d4147f
4,Consolato Italiano a Madrid,Government Building,Agustin De Betancourt 3,ES,Madrid,España,NaN,1498,"[Agustin De Betancourt 3, Madrid Madrid, España]","[{'label': 'display', 'lat': 40.43546789183154...",40.435468,-3.675468,NaN,Madrid,511e24e6e4b0f3711007ae7f
5,Liceo Italiano Enrico Fermi,School,NaN,ES,Madrid,España,NaN,2016,"[Madrid Madrid, España]","[{'label': 'display', 'lat': 40.4419026679497,...",40.441903,-3.695376,NaN,Madrid,4e310c00d1643f195b90e579
6,Cámara de Comercio e Industria Italiana para E...,Non-Profit,"Calle Cristobal Bordiú, 54",ES,Madrid,España,NaN,1969,"[Calle Cristobal Bordiú, 54, 28002 Madrid Madr...","[{'label': 'display', 'lat': 40.44274209763634...",40.442742,-3.695215,28002,Madrid,519dcef6454a4051f588105c
7,Camara de comercio italiana madrid,Non-Profit,"Calle Cristobal Bordiú, 54 MADRID",ES,Madrid,España,NaN,1963,"[Calle Cristobal Bordiú, 54 MADRID, 28002 Madr...","[{'label': 'display', 'lat': 40.44282374988759...",40.442824,-3.695183,28002,Madrid,519dcc9d454a4051f5866413
8,Café italiano,Internet Cafe,NaN,ES,Madrid,España,NaN,2386,"[Madrid Madrid, España]","[{'label': 'display', 'lat': 40.451143, 'lng':...",40.451143,-3.701401,NaN,Madrid,539a95a1498eda0a5f877242


In [31]:
dataframe_filtered_03.name


0    Embajada de la República Italiana - Ambasciata...
1                                     Mercato Italiano
2                                   Il Centro Italiano
3                                   Il Centro Italiano
4                          Consolato Italiano a Madrid
5                          Liceo Italiano Enrico Fermi
6    Cámara de Comercio e Industria Italiana para E...
7                   Camara de comercio italiana madrid
8                                        Café italiano
Name: name, dtype: object

In [32]:
venues_map_03 = folium.Map(location=[latitude_03, longitude_03], zoom_start=14) # generate map centred around the Conrad Hotel

# add a red circle marker to represent
folium.features.CircleMarker(
    [latitude_03, longitude_03],
    radius=10,
    color='red',
    popup='Our venue spot',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_03)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_03.lat, dataframe_filtered_03.lng, dataframe_filtered_03.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_03)

# display map
venues_map_03

In [ ]:
venues_map_03.save('v_03.html')

In [47]:
# C
dataframe_filtered_03.categories

0    Embassy / Consulate
1      Food & Drink Shop
2         Student Center
3         Student Center
4    Government Building
5                 School
6             Non-Profit
7             Non-Profit
8          Internet Cafe
Name: categories, dtype: object